<a href="https://colab.research.google.com/github/cku7808/NLP-practice/blob/main/LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSA(Latent Semantic Analysis, 잠재 의미 분석)**

TF-IDF와 Bag of Words는 토픽 기반이 아닌 단어 기반의 벡터이기 때문에 연관성이 있음에도 단어가 다르면 유사도가 0이 됨  
ex) hamberger, pizza -> 미국 음식 이라는 연관성이 있으나, 코사인 유사도 측정의 결과는 0

**그러나 LSA는 토픽 기반이기 때문에 유사도를 구할 수 있음**

# 선행 개념

**SVD(Singular Value Decomposition, 특이값 분해)**

SVD는 A가 m * n 행렬일 때, 다음과 같은 3개의 행렬의 곱으로 분해하는 것이다.  
$U : m*m$ 직교 행렬  
$V : n * n$ 직교 행렬  
$Σ : m * n$ 직사각 대각 행렬    





# *행렬 설명*

**직교 행렬** : `자신과 자신의 전치 행렬의 곱` or `자신의 전치 행렬과 자신의 곱`이 단위 행렬이 되는 행렬  
  
**단위 행렬(Identity Matrix)** : 주 대각선의 원소가 모두 1이고 나머지 원소는 모두 0인 정사각 행렬  
$ I= 
\left[\begin{matrix}
    1 & 0 & 0\\
    0 & 1 & 0\\
    0 & 0 & 1\\
\end{matrix}
\right] $  
  
*그런데 만약 행렬 A와 어떤 행렬의 곱이 단위 행렬(Identity Matrix)이라면 그 어떤 행렬은 행렬 A의 역행렬이다.  
$A \times A^{-1} = I$  

-> 직교 행렬 : $A \times A^T = I$를 만족하면서 $A^T \times A = I$를 동시에 만족하는 행렬 A  
그리고 역행렬의 정의에 따라 $A^T = A^{-1}$을 만족한다.



**대각 행렬** : 주 대각선을 제외한 나머지 원소가 0인 행렬  
ex) 3*3 대각 행렬  
$Σ = \left[\begin{matrix}
    a & 0 & 0\\
    0 & a & 0\\
    0 & 0 & a\\
\end{matrix}
\right] $  
  
  
그러나, m * n인 직사각 행렬인 경우는?  
$Σ = \left[\begin{matrix}
    a & 0 & 0 & 0\\
    0 & a & 0 & 0\\
    0 & 0 & a & 0\\
\end{matrix}
\right] $  
$Σ = \left[\begin{matrix}
    a & 0 & 0\\
    0 & a & 0\\
    0 & 0 & a\\
    0 & 0 & 0\\
\end{matrix}
\right] $  
위와 같은 형태를 갖는다.

# 다시 되돌아가서 ,,

SVD를 통해 나온 대각 행렬 $Σ$의 주대각원소를 행렬 A의 특이값이라고 부른다.  
그리고 이 특이값을 $σ_1,σ_2,...,σ_r$이라고 표현할 때, 특이값은 내림차순으로 정렬되어 있다!  
ex) $Σ = \left[\begin{matrix}
    12.4 & 0 & 0\\
    0 & 9.5 & 0\\
    0 & 0 & 1.3\\
\end{matrix}
\right] $  

앞서 설명한 SVD를 Full SVD라고 부르는데, LSA에서는 Full SVD를 사용하는 것이 아니라 3개의 행렬에서 일부 벡터들을 제외한 **절단된 SVD(Truncated SVD)**를 사용한다.  
<img src="https://drive.google.com/uc?id=193zdAmFey82kWABZLYl6nPb0UNdSHoKT">  
  
절단된 SVD는 대각 행렬 $Σ$의 주대각원소 중 상위값 t개만 보존되기 때문에 본래의 행렬 A로 복구할 수 없음  
* 여기서 t는 우리가 찾고자 하는 토픽의 개수를 의미한다  
* t를 크게 잡으면 기존 행렬의 정보를 더 많이 저장할 수 있지만, t를 작게 잡아야 노이즈가 줄어든다  
  
이렇게 일부 벡터들을 삭제하는 것을 **차원 축소**라고 부른다 -> 계산 비용 감소, 상대적으로 덜 중요한 정보 제거

# LSA

기존의 DTM, TF-IDF는 단어의 의미를 전혀 고려하지 않는다는 한계점을 지님  
그러나, LSA는 DTM/TF-IDF 행렬에 절단된 SVD를 사용하여 차원을 축소시킴으로써 단어들의 잠재된 의미를 끌어낼 수 있음

In [1]:
import numpy as np
from math import log

docs = [
    "먹고 싶은 사과",
    "먹고 싶은 바나나",
    "길고 노란 바나나 바나나",
    "저는 과일이 좋아요"
]
vocab = list(set(w for doc in docs for w in doc.split()))

vocab.sort()

In [2]:
# 총 문서의 개수 n
n = len(docs)

# TF 함수
def tf(t,doc):
  return doc.count(t)

In [3]:
# docs의 TF 구하기 -> DTM 만들기
DTM = []
for i in range(n):
  tmp = []
  for elem in vocab:
    tmp.append(tf(elem,docs[i]))
  DTM.append(tmp)

TF = np.array(DTM)
TF


array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

In [4]:
np.shape(TF)

(4, 9)

만들어진 4*9 DTM에 대해서 Full SVD를 수행

In [5]:
# 대각 행렬 시그마 : S
# np.linalg.svd는 대각행렬을 반환하는 것이 아닌 특이값 벡터를 반환
U, S, VT = np.linalg.svd(TF, full_matrices=True)

# 4*4 직교 행렬 U
print("행렬 U :")
print(U.round(2))
print("행렬 U의 크기 :",np.shape(U))

행렬 U :
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기 : (4, 4)


In [6]:
# 대각 행렬 S
print("특이값 벡터 :")
print(S.round(2))
print("특이값 벡터의 크기 :",np.shape(S))

특이값 벡터 :
[2.69 2.05 1.73 0.77]
특이값 벡터의 크기 : (4,)


np.linalg.svd가 대각 행렬이 아닌 특이값 벡터를 반환하기 때문에 대각 행렬을 만들어 줌  
np.diag는 입력값 n을 입력받아 1~n을 주대각원소로 갖는 대각 행렬을 만들어주는 함수  
S = np.diag(3) => $S = \left[\begin{matrix}
    1 & 0 & 0\\
    0 & 2 & 0\\
    0 & 0 & 3\\
\end{matrix}
\right]
$

In [7]:
# 대각 행렬의 크기 4*9의 임의의 행렬 생성
s = np.zeros((4,9))

# 특이값 삽입
s[:4,:4] = np.diag(S)

print("대각 행렬 s")
print(s.round(2))

print("대각 행렬의 크기 :",np.shape(s))

대각 행렬 s
[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
대각 행렬의 크기 : (4, 9)


In [8]:
# 직교 행렬 VT
print("직교 행렬 VT :")
print(VT.round(2))
print("직교 행렬 VT의 크기 :", np.shape(VT))

직교 행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
직교 행렬 VT의 크기 : (9, 9)


만들어진 4*9 DTM에 대하여 절단된 SVD(Truncated SVD) 수행

In [9]:
# 특이값 상위 2개만 보존 -> 찾고자 하는 토픽이 2개
s = s[:2,:2]
print("대각 행렬 s :")
print(s.round(2))

U = U[:,:2] #2개의 열만 남기고 제거 -> 2*2 대각 행렬에 맞춰 행렬의 곱을 수행할 수 있도록
print("직교 행렬 U :")
print(U.round(2))

VT = VT[:2,:] #2개의 행만 남기고 제거 -> 2*2 대각 행렬에 맞춰 행렬의 곱을 수행할 수 있도록
print("직교 행렬 U :")
print("직교 행렬 VT :")
print(VT.round(2))

대각 행렬 s :
[[2.69 0.  ]
 [0.   2.05]]
직교 행렬 U :
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]
직교 행렬 U :
직교 행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


차원 축소된 행렬 U*s*VT는 기존 행렬 TF와는 다른 행렬이다.  
축소된 행렬을 TF'이라 하고 두 행렬을 비교

In [11]:
TF_prime = np.dot(np.dot(U,s),VT)
print(TF)
print(TF_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


*차원이 축소된 U, s, VT의 의미*  



U : 축소된 U는 $4 \times 2$의 크기를 갖는데, 이는 문서의 개수 * 토픽 개수 t의 크기 와 같다  
-> 4개의 문서 각각을 2개의 값으로 표현  
-> U의 각 행은 잠재 의미 표현을 위해 수치화된 각각의 문서 벡터  

VT : 축소된 VT는 $2 \times 9$의 크기를 갖는데, 이는 토픽 개수 t * 단어의 개수 와 같다  
-> VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터

# LSA의 장단점

**장점**  
1. 쉽고 빠른 구현
2. 단어의 잠재적 의미 도출
3. 문서 유사도 계산

**단점**
1. SVD의 특성상 새로운 데이터를 추가하기 위해서는 처음부터 다시 계산해야함 -> 새로운 정보 업데이트 어려움